# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import tree
from sklearn.model_selection import cross_val_predict
from google.colab import files
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Carga de los data sets de test y de entrenamiento

In [ ]:
test = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Train_TP2_Datos_2020-2C.csv')

train.drop_duplicates(subset="Opportunity_ID", inplace=True)
test.drop_duplicates(subset="Opportunity_ID", inplace=True)

In [ ]:
train['Stage'].value_counts()

Closed Won       5072
Closed Lost      4719
Proposal           30
Negotiation        11
Qualification       9
Name: Stage, dtype: int64

Indicamos cuál es la columna a predecir, y con qué feature vamos a trabajar.

Para esta primer prueba, utilizamos el código burocrático 
de aprobación 0.

Dicho feature, según lo trabajado en el TP1, tenía influencia en el éxito final de la oportunidad, siendo mayor el éxito de las mismas en los casos en los que este código no era necesario.


In [ ]:
#X = DATA , Y = TARGET 
X1, y1 = train[['Bureaucratic_Code_0_Approval']], train['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

# XGBoost

Instanciamos el clasificador de XGBoost.

In [ ]:
xg_clas = XGBClassifier(colsample_bytree = 0.5, 
                        learning_rate = 0.2,
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 40)

Entrenamos el modelo.

In [ ]:
xg_clas.fit(X_train,y_train)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=40, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Predecimos una vez el modelo entrenado, y utilizando el set de validación.

In [ ]:
predicciones_train = xg_clas.predict_proba(X_test)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_train)

0.7053072483427356

Vemos las predicciones para el set de test de la competencia

In [ ]:
predicciones_finales = xg_clas.predict_proba(test['Bureaucratic_Code_0_Approval'].to_frame())

In [ ]:
predicciones_finales

array([[0.5799248 , 0.40365914, 0.00473217, 0.00782576, 0.00385809],
       [0.5799248 , 0.40365914, 0.00473217, 0.00782576, 0.00385809],
       [0.36549312, 0.62730575, 0.0024065 , 0.00234124, 0.00245335],
       ...,
       [0.5799248 , 0.40365914, 0.00473217, 0.00782576, 0.00385809],
       [0.36549312, 0.62730575, 0.0024065 , 0.00234124, 0.00245335],
       [0.36549312, 0.62730575, 0.0024065 , 0.00234124, 0.00245335]],
      dtype=float32)

Me quedo únicamente con la probabilidad de éxito.




In [ ]:
xg_clas.classes_

array(['Closed Lost', 'Closed Won', 'Negotiation', 'Proposal',
       'Qualification'], dtype=object)

Por cada registro, se estimo la probabilidad de cada uno de los 5 posibles Stages. Como se nos pide estimar la probabilidad de éxito, nos quedamos con el elemento 1 de cada uno de los arreglos. 

In [ ]:
predicciones_exito = []
for prob in predicciones_finales:
  predicciones_exito.append(prob[1])

In [ ]:
csv_kaggle = pd.DataFrame()
csv_kaggle['Opportunity_ID'] = test["Opportunity_ID"]
csv_kaggle['target'] = predicciones_exito

In [ ]:
csv = csv_kaggle.to_csv('predicciones_xgb.csv', index=False)
#files.download('predicciones_xgb.csv')

### Resultados

Algoritmo: **XGBoost**

Búsqueda de hiper parámetros: Aleatorio

Features analizados: 

- Bureaucratic Code 0 Approval

Hiper parámetros: 

- objective ='reg:squarederror'
- colsample_bytree = 0.5
- learning_rate = 0.2
- max_depth = 5
- alpha = 10
- n_estimators = 40

**Resultado Kaggle = 0,70119**

# Búsqueda de Hiperparámetros

Luego de realizar las predicciones con XGBoost, si bien obtuvimos buenos resultados, nos surge la pregunta de si podemos mejorarlos optimizando los hiperparámetros. Para buscar los óptimos, utilizamos Random Search. 

## Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

posibles_hiperparametros = {'learning_rate':np.arange(0.1,0.5,0.1),
                              'n_estimators':np.arange(16,116,15),
                              'scale_pos_weight':np.arange(2,6,1),
                              'max_depth':np.arange(4,12,1),
                            'min_child_weight':np.arange(1,10,1),
                              'gamma':np.arange(0,0.5,0.1),
                              'subsample':np.arange(0.6,1,0.1),
                            'colsample_bytree':np.arange(0.6,0.91,0.05),
                              'colsample_bylevel':np.arange(0.6,0.91,0.05)}

random_hiperparametros = RandomizedSearchCV(xg_clas, 
                                            posibles_hiperparametros, 
                                            n_iter = 100)
mejores_hp = random_hiperparametros.fit(X_train, y_train)

In [ ]:
mejores_hp.best_params_

{'colsample_bylevel': 0.9000000000000002,
 'colsample_bytree': 0.9000000000000002,
 'gamma': 0.0,
 'learning_rate': 0.30000000000000004,
 'max_depth': 11,
 'min_child_weight': 7,
 'n_estimators': 76,
 'scale_pos_weight': 3,
 'subsample': 0.7999999999999999}

In [ ]:
predicciones_mejores_hp_train = mejores_hp.predict_proba(X_test)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_mejores_hp_train)

0.7017893971647604

In [ ]:
predicciones_mejores_hp = mejores_hp.predict_proba(test['Bureaucratic_Code_0_Approval'].to_frame())

In [ ]:
predicciones_mejores_hp_exito = []
for prob in predicciones_mejores_hp:
  predicciones_mejores_hp_exito.append(prob[1])

In [ ]:
csv_kaggle_mejores_hp = pd.DataFrame()
csv_kaggle_mejores_hp['Opportunity_ID'] = test["Opportunity_ID"]
csv_kaggle_mejores_hp['target'] = predicciones_mejores_hp_exito

In [ ]:
csv = csv_kaggle_mejores_hp.to_csv('predicciones_xgb_randomSearch.csv', index=False)
#files.download('predicciones_xgb_randomSearch.csv')

### Resultados

Algoritmo: **XGBoost**

Búsqueda de hiper parámetros: **Random Search**

Features analizados: 

- Bureaucratic_Code_0_Approval

Hiper parámetros: 

- colsample_bylevel: 0.9000000000000002,
- colsample_bytree: 0.9000000000000002,
- gamma: 0.0,
- learning_rate: 0.4,
- max_depth: 8,
- min_child_weight: 1,
- n_estimators: 31,
- objective: 'binary:logistic',
- scale_pos_weight: 3,
- subsample: 0.7

**Resultado Kaggle = 0,70119**

La mejora con respecto a los hiper parámetros propuestos antes de esta búsqueda son demasiado leves, y no hacen diferencia en Kaggle. Probamos haciendo random search con 1000 iteraciones, pero la celda nunca terminaba de correr.